In [1]:
import numpy as np
import pandas as pd
import numpy.matlib
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


In [2]:
policy_data=pd.read_csv("https://raw.githubusercontent.com/OxCGRT/USA-covid-policy/master/data/OxCGRT_US_latest.csv")
mobility_data=pd.read_csv("/Users/Tara/Downloads/Global_Mobility_Report (4).csv")


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
policy_data


,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C1_Notes,C2_Workplace closing,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,United States,USA,NaN,NaN,NAT_GOV,20200101,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
1,United States,USA,NaN,NaN,NAT_GOV,20200102,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
2,United States,USA,NaN,NaN,NAT_GOV,20200103,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
3,United States,USA,NaN,NaN,NAT_GOV,20200104,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
4,United States,USA,NaN,NaN,NAT_GOV,20200105,0.0,NaN,NaN,0.0,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13828,United States,USA,Wyoming,US_WY,STATE_ALL,20200913,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0
13829,United States,USA,Wyoming,US_WY,STATE_ALL,20200914,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0
13830,United States,USA,Wyoming,US_WY,STATE_ALL,20200915,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0
13831,United States,USA,Wyoming,US_WY,STATE_ALL,20200916,NaN,NaN,NaN,NaN,...,NaN,37.04,NaN,47.62,NaN,46.15,NaN,45.45,NaN,50.0


In [5]:
#     # getting rid of the months of january, february, and september from policy data,
#     no_jan_policy_data=~policy_data.Date.astype(str).str.contains("202001")
#     policy_data=policy_data[no_jan_policy_data]
    
#     no_feb_policy_data=~policy_data.Date.astype(str).str.contains("202002")
#     policy_data=policy_data[no_feb_policy_data]
    
sept_policy_data=policy_data.Date.astype(str).str.contains("202009")
policy_data=policy_data[sept_policy_data]


In [6]:
# getting rid of US virgin islands from policy data
no_usvi_policy_data=~policy_data.RegionName.astype(str).str.contains("Virgin Islands")
policy_data=policy_data[no_usvi_policy_data]


In [7]:
# further cleaning the policy data
policy_data=policy_data[["RegionName", "Jurisdiction", "Date", 'StringencyIndexForDisplay', 'GovernmentResponseIndexForDisplay', 'ContainmentHealthIndexForDisplay', 'EconomicSupportIndexForDisplay']]
policy_data=policy_data[policy_data.Jurisdiction=="STATE_ALL"]
policy_data.drop('Jurisdiction', axis=1, inplace=True)
policy_data.dropna(axis=0, how="any", inplace=True)


In [8]:
policy_data

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay
505,Alaska,20200901,47.69,59.94,57.20,75.0
506,Alaska,20200902,47.69,59.94,57.20,75.0
507,Alaska,20200903,47.69,59.94,57.20,75.0
508,Alaska,20200904,47.69,59.94,57.20,75.0
509,Alaska,20200905,47.69,59.94,57.20,75.0
...,...,...,...,...,...,...
13828,Wyoming,20200913,37.04,46.15,45.45,50.0
13829,Wyoming,20200914,37.04,46.15,45.45,50.0
13830,Wyoming,20200915,37.04,46.15,45.45,50.0
13831,Wyoming,20200916,37.04,46.15,45.45,50.0


In [9]:
mobility_data

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441512,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-07,NaN,NaN,NaN,NaN,-9.0,NaN
2441513,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-08,NaN,NaN,NaN,NaN,-1.0,NaN
2441514,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-09,NaN,NaN,NaN,NaN,2.0,NaN
2441515,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2020-09-10,NaN,NaN,NaN,NaN,-7.0,NaN


In [10]:
# filtering mobility data so it only includes the US
mobility_data=mobility_data[mobility_data.country_region=="United States"]

# # getting rid of the months of february, and september from mobility data
# no_feb_mobility_data=~mobility_data.date.astype(str).str.contains("2020-02")
# mobility_data=mobility_data[no_feb_mobility_data]

sept_mobility_data=mobility_data.date.astype(str).str.contains("2020-09")
mobility_data=mobility_data[sept_mobility_data]

mobility_data


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
1860677,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-01,-9.0,1.0,59.0,-31.0,-36.0,9.0
1860678,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-02,-9.0,-2.0,59.0,-31.0,-36.0,9.0
1860679,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-03,-8.0,2.0,70.0,-28.0,-36.0,8.0
1860680,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-04,-9.0,0.0,80.0,-25.0,-37.0,8.0
1860681,US,United States,NaN,NaN,NaN,NaN,NaN,2020-09-05,-12.0,3.0,108.0,-21.0,-17.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416832,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-09-07,NaN,NaN,NaN,NaN,-61.0,NaN
2416833,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-09-08,NaN,NaN,NaN,NaN,-20.0,NaN
2416834,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-09-09,NaN,NaN,NaN,NaN,-10.0,NaN
2416835,US,United States,Wyoming,Weston County,NaN,NaN,56045.0,2020-09-10,NaN,NaN,NaN,NaN,-12.0,NaN


In [11]:
# further cleaning the mobility data
# mobility_data.dropna(axis=0, how="any", inplace=True)
mobility_data=mobility_data[["country_region", "sub_region_1", "date", 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"]]
mobility_data.dropna(axis=0, how="any", inplace=True)


<ipython-input-11-5c789d4562c4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mobility_data.dropna(axis=0, how="any", inplace=True)


In [12]:
mobility_data

,country_region,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
1860887,United States,Alabama,2020-09-01,-2.0,6.0,-2.0,-27.0,6.0,53.0
1860888,United States,Alabama,2020-09-02,-1.0,2.0,-1.0,-28.0,6.0,73.0
1860889,United States,Alabama,2020-09-03,1.0,8.0,5.0,-27.0,5.0,69.0
1860890,United States,Alabama,2020-09-04,-1.0,4.0,10.0,-30.0,5.0,77.0
1860891,United States,Alabama,2020-09-05,-5.0,9.0,12.0,-17.0,1.0,115.0
...,...,...,...,...,...,...,...,...,...
2415145,United States,Wyoming,2020-09-04,0.0,22.0,19.0,-28.0,1.0,125.0
2415149,United States,Wyoming,2020-09-08,-6.0,3.0,-5.0,-30.0,8.0,25.0
2415150,United States,Wyoming,2020-09-09,4.0,15.0,1.0,-25.0,4.0,62.0
2415151,United States,Wyoming,2020-09-10,6.0,13.0,13.0,-26.0,2.0,85.0


In [13]:
mobility_data_states=mobility_data.sub_region_1.unique()
mobility_data_dates=mobility_data.date.unique()
mobility_data_states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [14]:
# # condensing mobility data (putting it into a new df) so there's 1 data point per day per state,
# mobility_data_condensed=pd.DataFrame(columns=["RegionName", "Date", "Retail_and_Recreational", "Grocery_and_Pharmacy", "Transit_Stations", "Workplaces", "Residential", "Parks"])

# dict_of_rows_to_average={}
# for index, row in mobility_data.iterrows():
#     for i in mobility_data_dates:
#         for j in mobility_data_states:
#             str_lookup=str(i)+str(j)
#             if str_lookup in dict_of_rows_to_average:
#                 dict_of_rows_to_average[str_lookup]=dict_of_rows_to_average[str_lookup].append(row)
#             else:
#                 dict_of_rows_to_average[str_lookup]=pd.DataFrame(columns=["country_region", "sub_region_1", "date", 'retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline', "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"])
#                 dict_of_rows_to_average[str_lookup]=dict_of_rows_to_average[str_lookup].append(row)


In [16]:
mobility_data2 = mobility_data.groupby(['sub_region_1','date'])['retail_and_recreation_percent_change_from_baseline', "grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"].apply(np.mean).reset_index()


<ipython-input-16-1944d576671d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mobility_data2 = mobility_data.groupby(['sub_region_1','date'])['retail_and_recreation_percent_change_from_baseline', "grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline"].apply(np.mean).reset_index()


In [17]:
mobility_data2

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alabama,2020-09-01,-2.500000,5.166667,-9.500000,-27.166667,6.166667,47.833333
1,Alabama,2020-09-02,-2.857143,2.142857,-4.285714,-28.714286,6.142857,58.714286
2,Alabama,2020-09-03,-0.600000,8.200000,1.200000,-28.400000,5.800000,53.000000
3,Alabama,2020-09-04,-1.555556,4.111111,2.111111,-30.333333,5.111111,55.888889
4,Alabama,2020-09-05,-3.666667,11.111111,8.444444,-17.222222,0.555556,95.333333
...,...,...,...,...,...,...,...,...
545,Wyoming,2020-09-07,8.000000,24.000000,34.000000,-71.000000,14.000000,119.000000
546,Wyoming,2020-09-08,-8.000000,2.666667,-15.333333,-30.333333,8.333333,66.333333
547,Wyoming,2020-09-09,2.333333,11.666667,4.333333,-24.666667,4.333333,113.333333
548,Wyoming,2020-09-10,10.000000,16.500000,20.500000,-24.500000,1.500000,177.500000


In [18]:
# print(mobility_data2.to_string())

In [19]:
# mobility_data[mobility_data["sub_region_1"] == "Alabama"][mobility_data["date"] == "2020-03-01"]

In [20]:
mobility_data=mobility_data2
mobility_data

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alabama,2020-09-01,-2.500000,5.166667,-9.500000,-27.166667,6.166667,47.833333
1,Alabama,2020-09-02,-2.857143,2.142857,-4.285714,-28.714286,6.142857,58.714286
2,Alabama,2020-09-03,-0.600000,8.200000,1.200000,-28.400000,5.800000,53.000000
3,Alabama,2020-09-04,-1.555556,4.111111,2.111111,-30.333333,5.111111,55.888889
4,Alabama,2020-09-05,-3.666667,11.111111,8.444444,-17.222222,0.555556,95.333333
...,...,...,...,...,...,...,...,...
545,Wyoming,2020-09-07,8.000000,24.000000,34.000000,-71.000000,14.000000,119.000000
546,Wyoming,2020-09-08,-8.000000,2.666667,-15.333333,-30.333333,8.333333,66.333333
547,Wyoming,2020-09-09,2.333333,11.666667,4.333333,-24.666667,4.333333,113.333333
548,Wyoming,2020-09-10,10.000000,16.500000,20.500000,-24.500000,1.500000,177.500000


In [21]:
mobility_data['sub_region_1'] = np.where(mobility_data['sub_region_1'] == 'District of Columbia', 'Washington DC', mobility_data['sub_region_1'])
mobility_data["state_date_id"] = mobility_data.apply(lambda row: row.sub_region_1 + '_' + row.date.replace('-',''), axis=1)


In [22]:
policy_data["state_date_id"] = policy_data.apply(lambda row: row.RegionName + '_' + str(row.Date), axis=1)

In [23]:
mobility_data

,sub_region_1,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline,state_date_id
0,Alabama,2020-09-01,-2.500000,5.166667,-9.500000,-27.166667,6.166667,47.833333,Alabama_20200901
1,Alabama,2020-09-02,-2.857143,2.142857,-4.285714,-28.714286,6.142857,58.714286,Alabama_20200902
2,Alabama,2020-09-03,-0.600000,8.200000,1.200000,-28.400000,5.800000,53.000000,Alabama_20200903
3,Alabama,2020-09-04,-1.555556,4.111111,2.111111,-30.333333,5.111111,55.888889,Alabama_20200904
4,Alabama,2020-09-05,-3.666667,11.111111,8.444444,-17.222222,0.555556,95.333333,Alabama_20200905
...,...,...,...,...,...,...,...,...,...
545,Wyoming,2020-09-07,8.000000,24.000000,34.000000,-71.000000,14.000000,119.000000,Wyoming_20200907
546,Wyoming,2020-09-08,-8.000000,2.666667,-15.333333,-30.333333,8.333333,66.333333,Wyoming_20200908
547,Wyoming,2020-09-09,2.333333,11.666667,4.333333,-24.666667,4.333333,113.333333,Wyoming_20200909
548,Wyoming,2020-09-10,10.000000,16.500000,20.500000,-24.500000,1.500000,177.500000,Wyoming_20200910


In [47]:
num=0
for index, row in mobility_data.iterrows():
    if row["sub_region_1"]=="New York":
        num+=1
print(num)

11


In [24]:
policy_data

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,state_date_id
505,Alaska,20200901,47.69,59.94,57.20,75.0,Alaska_20200901
506,Alaska,20200902,47.69,59.94,57.20,75.0,Alaska_20200902
507,Alaska,20200903,47.69,59.94,57.20,75.0,Alaska_20200903
508,Alaska,20200904,47.69,59.94,57.20,75.0,Alaska_20200904
509,Alaska,20200905,47.69,59.94,57.20,75.0,Alaska_20200905
...,...,...,...,...,...,...,...
13828,Wyoming,20200913,37.04,46.15,45.45,50.0,Wyoming_20200913
13829,Wyoming,20200914,37.04,46.15,45.45,50.0,Wyoming_20200914
13830,Wyoming,20200915,37.04,46.15,45.45,50.0,Wyoming_20200915
13831,Wyoming,20200916,37.04,46.15,45.45,50.0,Wyoming_20200916


In [48]:
num=0
for index, row in mobility_data.iterrows():
    if row["sub_region_1"]=="New York":
        num+=1
print(num)

11


In [53]:
inner_join_df=pd.merge(policy_data[["state_date_id", "RegionName", "Date", "StringencyIndexForDisplay", "GovernmentResponseIndexForDisplay", "ContainmentHealthIndexForDisplay", "EconomicSupportIndexForDisplay"]], mobility_data[["state_date_id", "retail_and_recreation_percent_change_from_baseline","grocery_and_pharmacy_percent_change_from_baseline", "transit_stations_percent_change_from_baseline", "workplaces_percent_change_from_baseline", "residential_percent_change_from_baseline", "parks_percent_change_from_baseline" ]], on='state_date_id', how='inner')
inner_join_df


,state_date_id,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alaska_20200901,Alaska,20200901,47.69,59.94,57.20,75.0,11.000000,14.500000,-11.000000,-30.000000,8.000000,109.000000
1,Alaska_20200902,Alaska,20200902,47.69,59.94,57.20,75.0,7.000000,7.000000,-12.000000,-30.500000,8.000000,86.000000
2,Alaska_20200903,Alaska,20200903,47.69,59.94,57.20,75.0,6.500000,10.000000,-10.000000,-30.000000,7.500000,120.500000
3,Alaska_20200904,Alaska,20200904,47.69,59.94,57.20,75.0,4.000000,9.000000,-9.000000,-34.500000,6.500000,148.500000
4,Alaska_20200905,Alaska,20200905,47.69,59.94,57.20,75.0,0.500000,10.500000,-2.500000,-17.500000,-1.000000,205.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Wyoming_20200907,Wyoming,20200907,37.04,46.15,45.45,50.0,8.000000,24.000000,34.000000,-71.000000,14.000000,119.000000
490,Wyoming_20200908,Wyoming,20200908,37.04,46.15,45.45,50.0,-8.000000,2.666667,-15.333333,-30.333333,8.333333,66.333333
491,Wyoming_20200909,Wyoming,20200909,37.04,46.15,45.45,50.0,2.333333,11.666667,4.333333,-24.666667,4.333333,113.333333
492,Wyoming_20200910,Wyoming,20200910,37.04,46.15,45.45,50.0,10.000000,16.500000,20.500000,-24.500000,1.500000,177.500000


In [54]:
inner_join_df.drop(["state_date_id"], axis=1,inplace=True)

In [55]:
inner_join_df

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
0,Alaska,20200901,47.69,59.94,57.20,75.0,11.000000,14.500000,-11.000000,-30.000000,8.000000,109.000000
1,Alaska,20200902,47.69,59.94,57.20,75.0,7.000000,7.000000,-12.000000,-30.500000,8.000000,86.000000
2,Alaska,20200903,47.69,59.94,57.20,75.0,6.500000,10.000000,-10.000000,-30.000000,7.500000,120.500000
3,Alaska,20200904,47.69,59.94,57.20,75.0,4.000000,9.000000,-9.000000,-34.500000,6.500000,148.500000
4,Alaska,20200905,47.69,59.94,57.20,75.0,0.500000,10.500000,-2.500000,-17.500000,-1.000000,205.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
489,Wyoming,20200907,37.04,46.15,45.45,50.0,8.000000,24.000000,34.000000,-71.000000,14.000000,119.000000
490,Wyoming,20200908,37.04,46.15,45.45,50.0,-8.000000,2.666667,-15.333333,-30.333333,8.333333,66.333333
491,Wyoming,20200909,37.04,46.15,45.45,50.0,2.333333,11.666667,4.333333,-24.666667,4.333333,113.333333
492,Wyoming,20200910,37.04,46.15,45.45,50.0,10.000000,16.500000,20.500000,-24.500000,1.500000,177.500000


In [57]:
num=0
for index, row in inner_join_df.iterrows():
    if row["RegionName"]=="Connecticut":
        num+=1
print(num)

11


In [64]:
CTdata=inner_join_df[inner_join_df.RegionName=="Connecticut"]
CTdata=CTdata[9:]

In [65]:
CTdata

,RegionName,Date,StringencyIndexForDisplay,GovernmentResponseIndexForDisplay,ContainmentHealthIndexForDisplay,EconomicSupportIndexForDisplay,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,parks_percent_change_from_baseline
75,Connecticut,20200910,56.48,66.03,64.39,75.0,-6.20,-2.00,-21.2,-35.80,9.40,61.4
76,Connecticut,20200911,56.48,66.03,64.39,75.0,-4.75,-2.75,-16.5,-33.25,6.75,120.0


In [67]:
CTdata.to_csv('/Users/Tara/Downloads/september_CT_data.csv', header=True)